In [1]:
import pandas as pd
from pandas import DataFrame
import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader, random_split
# from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import math
import numpy as np

import nni
import torch.nn.functional as F
import nni.retiarii.nn.pytorch as nn
from nni.retiarii import model_wrapper

from os.path import exists

from collections import OrderedDict

# import ray
# from ray import tune # for trialing
# from ray.tune import JupyterNotebookReporter # for trial reporting
# # from ray.tune.integration.torch import is_distributed_trainable
# # from torch.nn.parallel import DistributedDataParallel
# # from ray.tune.integration.torch import DistributedTrainableCreator
# # from ray.tune.integration.torch import distributed_checkpoint_dir
# from ray.tune.schedulers import ASHAScheduler # for trial scheduling
# from ray.tune.schedulers import AsyncHyperBandScheduler
# from ray.tune.suggest.bayesopt import BayesOptSearch

In [2]:
# # Import Kaggle data
# import kaggle
# kagggle competitions download tabular-playground-series-mar-2021

In [3]:
# Add data to memory for discovery
train,test = pd.read_csv('./train.csv',index_col="id"),pd.read_csv('./test.csv',index_col="id")

In [4]:
# Get data structure
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300000 entries, 0 to 499999
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cat0    300000 non-null  object 
 1   cat1    300000 non-null  object 
 2   cat2    300000 non-null  object 
 3   cat3    300000 non-null  object 
 4   cat4    300000 non-null  object 
 5   cat5    300000 non-null  object 
 6   cat6    300000 non-null  object 
 7   cat7    300000 non-null  object 
 8   cat8    300000 non-null  object 
 9   cat9    300000 non-null  object 
 10  cat10   300000 non-null  object 
 11  cat11   300000 non-null  object 
 12  cat12   300000 non-null  object 
 13  cat13   300000 non-null  object 
 14  cat14   300000 non-null  object 
 15  cat15   300000 non-null  object 
 16  cat16   300000 non-null  object 
 17  cat17   300000 non-null  object 
 18  cat18   300000 non-null  object 
 19  cont0   300000 non-null  float64
 20  cont1   300000 non-null  float64
 21  cont2   30

In [5]:
# Peek at data examples
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
id,,,,,,,,,,,,,,,,,,,,,
0,A,I,A,B,B,BI,A,S,Q,A,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,A,I,A,A,E,BI,K,W,AD,F,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,A,K,A,A,E,BI,A,E,BM,L,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,A,K,A,C,E,BI,A,Y,AD,F,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,A,I,G,B,E,BI,C,G,Q,A,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [6]:
# Get number of unique categories
cal_cols = train.select_dtypes('object')
cal_cols_keys = cal_cols.keys()
# cal_cols_vals = np.array((cal_cols.values))

#Build sample item
cat_stats = np.array([train[c].nunique() for c in cal_cols])
median_cat = math.ceil(np.median(cat_stats))
target_cat_index = int(np.where(cat_stats == median_cat)[0])
target_cat_name = cal_cols_keys[target_cat_index]

print("Categroy with median total unique values:\n",target_cat_name,": ",train[target_cat_name].nunique(),"unique values")

Categroy with median total unique values:
 cat3 :  13 unique values


In [7]:
# Index categories
categoricals_to_ix = {}

for c in train.select_dtypes('object'):
    categoricals_to_ix[c] = {cat:i for i,cat in enumerate(train[c].unique())}

#Confirm with sample indexing
sample_cat = categoricals_to_ix[target_cat_name]
print(target_cat_name,"indices:\n",sample_cat)



cat3 indices:
 {'B': 0, 'A': 1, 'C': 2, 'D': 3, 'G': 4, 'N': 5, 'H': 6, 'F': 7, 'E': 8, 'K': 9, 'I': 10, 'J': 11, 'L': 12}


In [8]:
# Test embeddings
test_embeds = {}

for cix_key in categoricals_to_ix.keys():
    cix = categoricals_to_ix[cix_key]
    bag_size = len(cix)
    embed_size = math.isqrt(bag_size) #hyperparameter to optimize later
    test_embeds[cix_key] = nn.Embedding(bag_size,embed_size)

#Confirm with sample embedding
target_cat_keys = list(sample_cat.keys())
target_cat_vals = np.array(list(sample_cat.values()))
target_cat_vals_median = math.ceil(np.median(target_cat_vals))
target_cat_keys_sample = target_cat_keys[target_cat_vals_median]
print("Sanple",target_cat_name,"category:\n",target_cat_keys_sample)
    
testor = torch.tensor([categoricals_to_ix[target_cat_name][target_cat_keys_sample]], dtype=torch.long)
cat_keys_sample_hash = test_embeds[target_cat_name](testor)
print("Sanple category embedding:\n",cat_keys_sample_hash)

Sanple cat3 category:
 H
Sanple category embedding:
 tensor([[-0.3307,  0.5979,  0.5125]], grad_fn=<EmbeddingBackward>)


In [9]:
# Define embeddings content (dictionary and translations)
#function to build embeddings library
#function assumes only categorical features are fed in
def BuildEmbedlibrary(features):
    categoricals_to_ix,embeds = {},{}
    
    #Build category index
    for c in features:
        categoricals_to_ix[c] = {cat:i for i,cat in enumerate(train[c].unique())}
    
    #Create embeddings for each feature
    for cix_key in categoricals_to_ix.keys():
        cix = categoricals_to_ix[cix_key]
        bag_size = len(cix)
        embed_size = math.isqrt(bag_size) #hyperparameter to optimize later
        embeds[cix_key] = nn.Embedding(bag_size,embed_size)
    
#         print(embeds[cix_key])
    return categoricals_to_ix,embeds

#function that gets respective embedding for given feature value
def CatToSubvector(category,feature_name,categorical_library,embedding_library,verbose=False):
    if verbose:
#         print(verbose)
#         for info in [categorical_library]:
# #         for info in [category,feature_name,categorical_library,embedding_library]:
#             print(info,"\n")
        print("category: {}; feature_name: {}; categorical_library: {}; embedding_library: {}".format(
            category,
            feature_name,
            categorical_library,
            embedding_library
        ))
#     [print(valu,"\n") for valu in [category,feature_name,categorical_library,embedding_library]]
    raw_idx = torch.tensor([categorical_library[feature_name][category]], dtype=torch.long)
#     if (verbose): print("embedding_library[feature_name]:",embedding_library[feature_name],"\n","raw_idx:",raw_idx)
    emb = embedding_library[feature_name](raw_idx)
#     if (verbose): print("embedding_library[feature_name](raw_idx):",emb,"\n")
#     subvector = torch.Tensor(emb)
#     if (verbose): print("subvector:",subvector)
    
#     return subvector
    return emb

In [10]:
# # Confirm function structure
test_cat_dict,test_embeddings = BuildEmbedlibrary(train.select_dtypes('object'))
test_subvec = CatToSubvector(target_cat_keys_sample,target_cat_name,test_cat_dict,test_embeddings)

print("Embedding Test:")
print("\tSanple feature:",target_cat_name)
print("\tSanple category:",target_cat_keys_sample)
print("\tSanple category embedding:",test_subvec)

Embedding Test:
	Sanple feature: cat3
	Sanple category: H
	Sanple category embedding: tensor([[0.0620, 0.9258, 0.0037]], grad_fn=<EmbeddingBackward>)


In [11]:
# Define data

#define dataset
class ClaimsDataset(Dataset):
    def __init__(self, csv_path,labelled=False):
        """
        Args:
            raw_data (dataframe): Dataframe with raw featurres and labels.
        """
        file_check_target = "train" if (labelled) else "test"
        
#         print("{}_set_x.pt".format(file_check_target))
#         print("{}_set_y.pt".format(file_check_target))
        
        if(exists( "{}_set_x.pt".format(file_check_target) ) and exists( "{}_set_y.pt".format(file_check_target) )):
            self.data = torch.load("{}_set_x.pt".format(file_check_target))
            self.target = torch.load("{}_set_y.pt".format(file_check_target))
        else:
            # Read data
            raw_data = pd.read_csv(csv_path)

    #         self.l = len(raw_data)
            # Separate future parameters/labels
            labels = raw_data.pop('target') if labelled else torch.tensor([], dtype=torch.long)
            categorical_features = raw_data.select_dtypes(include='object')
            noncategorical_features = raw_data.select_dtypes(exclude='object')

            # Build feature embeddings
            categoricals,embeddings = BuildEmbedlibrary(categorical_features)

            # Convert embeddings to parameters
            subvects = DataFrame()
    #         for row in categorical_features.values:
    #             subsubvects = []
    #             for v,c in zip(row,categorical_features.columns):
    #                 subsubvects.append(CatToSubvector(v,c,self.categoricals,self.embeddings))
    # #             [print("subsubvect size:",ssv.size()) for ssv in subsubvects]
    #             subvects.append(torch.cat(subsubvects,1))
            subsubvects = []

            for col in categorical_features.columns:
                print("Converting collection of categories, {}, to collection of embeddings...".format(col))
                cat_col = (categorical_features[col]).tolist()
    #             print("cat_col:",cat_col,"\n")
    #             cat_emb = cat_col
    #             cat_emb = cat_col.apply(lambda x: CatToSubvector(x,col,self.categoricals,self.embeddings,True))
    #             if (i%2):# == 0 and i!=0):
    #                 print("i:",i)
    #                 cat_emb = cat_col.apply(lambda x: CatToSubvector(x,col,self.categoricals,self.embeddings,True))
    #                 print("cat_emb:",cat_emb,"\n")
    #             else:
    #                 cat_emb = cat_col.apply(lambda x: CatToSubvector(x,col,self.categoricals,self.embeddings,False))
                cat_embs = CatToSubvector(cat_col[0],col,categoricals,embeddings,False)
                for x in cat_col[1:]:
                    new_emb = CatToSubvector(x,col,categoricals,embeddings,False)
                    cat_embs = torch.cat((cat_embs,new_emb))
    #             cat_emb = [CatToSubvector(x,col,self.categoricals,self.embeddings) for x in cat_col]
    #             print(col,"cat_embs.shape:",cat_embs.shape,"\n")
    #             cat_emb = torch.cat(tuple(cat_emb))
    #             print("type(cat_emb[-10]):",type(cat_emb[-10]),"\n")
    #             print("cat_emb[-10]:",cat_emb[-10],"\n")
                subsubvects.append(cat_embs)

    #         print("tuple(subsubvects):",tuple(subsubvects),"\n")
            subvects = torch.cat(tuple(subsubvects),1)
    #         print("subvects[1]:",subvects[1],"\n")
    #         print("subvects.shape:",subvects.shape,"\n")



            # copy the data 
            df_min_max_scaled = noncategorical_features.copy()

            # apply normalization techniques by Column 1 
            for col in df_min_max_scaled.columns:
                df_min_max_scaled[col] = (df_min_max_scaled[col] - df_min_max_scaled[col].min()) /\
                    (df_min_max_scaled[col].max() - df_min_max_scaled[col].min())

    #         noncat_mean = noncategorical_features.mean(1)
    #         noncat_std = noncategorical_features.std(1)


            self.data = torch.cat((
                subvects,
    #             torch.tensor(noncategorical_features.values)
                torch.tensor(df_min_max_scaled.values)
            ),1)
            torch.save(self.data,"{}_set_x.pt".format(file_check_target))

    #         print("labels:",labels)
    #         print("labels.tolist():",labels.tolist())
            self.target = torch.tensor(labels.tolist(), dtype=torch.int)
            torch.save(self.target,"{}_set_y.pt".format(file_check_target))
    #         print("self.target:",self.target)
                
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
            
        return x,y
    
    def __len__(self):
        return len(self.data)

In [12]:
##TODO add functionality for unlabbeled data
def LoadData(bs=4,target='train'):
    sets = {
        'train':None,
        'test':None,
    }
    train_loader,test_loader = None,None
#     bs = 2*batch_size_oom
    
    if(target=='train'):
        sets['train'] = ClaimsDataset('./train.csv',labelled=True)
        train_loader = DataLoader(sets['train'],batch_size=bs,shuffle=True)
    elif(target=='test'):
        sets['test'] = ClaimsDataset('./test.csv')
        test_loader = DataLoader(sets['test'],batch_size=bs,shuffle=True)
    elif(target=='both'):
        sets['train'],sets['test'] = ClaimsDataset('./train.csv',labelled=True),ClaimsDataset('./test.csv')
        train_loader,test_loader = DataLoader(sets['train'],batch_size=bs,shuffle=True),DataLoader(sets['test'],batch_size=bs,shuffle=True)
    else:
        print("Invalid target name supplied. Returning None.")
    
#     train_set,test_set = ClaimsDataset('./train.csv',labelled=True),ClaimsDataset('./test.csv')
#     train_loader,test_loader = DataLoader(claimset,batch_size=bs,shuffle=True)
#     train_loader,test_loader = DataLoader(train_set,batch_size=bs,shuffle=True),DataLoader(test_set,batch_size=bs,shuffle=True)
    
    return train_loader,test_loader

In [13]:
trainloader,testloader = LoadData()
# # print("cd.categoricals:",cd.categoricals,"\n","cd.embeddings:",cd.embeddings,"\n","cd.vectors.shape:",cd.vectors.shape)
i = 0
for x,y in trainloader:
    if not i%25000:
        print("i:",i)
        print("x:",x)
        print("y:",y)

    i+=1

i: 0
x: tensor([[-0.6965, -0.2979,  0.4077, -0.6213, -0.4913, -2.4970, -1.6142, -0.5667,
          0.8375,  0.0085,  0.2313, -0.6016, -0.2974,  1.8078,  1.0748, -0.3439,
          0.7384,  1.5573, -0.8074, -1.6828, -0.5794,  0.5368, -0.3502,  1.4539,
          0.6111, -0.4715, -1.4321,  0.0653,  0.5138,  0.3910,  0.3856,  1.0835,
         -0.0882,  0.1051,  1.2432,  0.2977,  0.6710, -0.8622,  2.5038,  0.4307,
         -1.2018, -0.8268, -0.3651,  0.2640,  1.1973,  0.2725, -0.1083, -0.0272,
          0.1926, -2.5174, -0.2493,  0.7241,  0.1441, -1.6702,  0.1370,  0.2136,
         -1.3672,  1.0691,  0.8435, -1.5849, -1.0602,  0.6213, -0.0863,  0.1523,
         -0.3435, -0.6364,  1.2252,  0.0473,  0.7524,  0.3026, -1.2232,  1.3522,
          0.7104, -1.8895,  1.1652,  0.6810,  0.6258,  0.7782,  0.7351,  0.8221,
          0.7600,  0.1263,  0.3402,  0.8293,  0.7478,  0.9246,  0.6246],
        [ 0.2649, -0.5148, -1.7510, -0.1913, -0.5488,  1.3885,  0.6922,  0.3423,
          0.7208, -0.0920, -

In [14]:
print(trainloader.dataset.data.shape)

torch.Size([300000, 87])


# train_set_x = (trainloader.dataset)[:][0]
r = torch.load('train_set_x.pt')
print(r[200000:200004])
# train_set_y = (trainloader.dataset)[:][1]
e = torch.load('train_set_y.pt')
print(e[200000:200004])
# numpy.save('data/train_set_x.pt',(trainloader.dataset)[:][0])

d = OrderedDict()
d[1] = "hi"
d["2"] = "hi"

len(d)

In [18]:
# # Define model search space
# import torch.nn.functional as F
# import nni.retiarii.nn.pytorch as nn
# from nni.retiarii import model_wrapper

class FullConn(nn.Module):
    def __init__(self,signal):
        out_features = nn.ValueChoice(range(2**10+1))
        num_groups = nn.ValueChoice(range(2, o_f/2+1))
        p = nn.ValueChoice(range(0.5,0.91,0.01))
        
        self.linear = nn.Linear(
            in_features = signal.shape[-1],
            out_features = out_features
        )
        self.norm = nn.GroupNorm(
            num_groups = num_groups,
            num_channels = o_f
        )        
        self.use_dropout = nn.ValueChoice([0,1])
        if (self.use_dropout):
            self.drop = nn.Dropout(p = p)
    def forward(self, x):
        x = self.linear(x)
        x = self.norm(x)
        x = F.leaky_relu(x)
        if (self.use_dropout): x = self.dropout(x)
        return x

class Soft(nn.Module):
    def __init__(self):
        self.softmax = nn.LogSoftmax()
    def forward(self, x):
        return self.softmax(x)

    
class Net(nn.Module):
    def __init__(self,signal_size,output_size):
        super().__init__()
#         signal_tensor = torch.load('train_set_x.pt')
        stack = OrderedDict()
        layer_count = nn.ValueChoice(range(2,2**10+1))
#         signal = signal_tensor.shape[-1]
#         i = 0
        for l in range(layer_count):
#             index = "{}".format(i)
            stack[len(stack)] = FullConn(signal_size)
#             i += 1
#             stack[len(stack)] = Drop()
        stack[len(stack)] = Soft()
        self.fc = nn.Sequential(stack)

@model_wrapper      # this decorator should be put on the out most PyTorch module
class Net(nn.Module):
    def __init__(self,signal_size,output_size):
        super().__init__()
#         signal_tensor = torch.load('train_set_x.pt')
        stack = OrderedDict()
        layer_count = nn.ValueChoice(range(2,2**10+1))
#         signal = signal_tensor.shape[-1]
#         i = 0
        for l in range(layer_count):
#             index = "{}".format(i)
            stack[len(stack)] = FullConn(signal_size)
#             i += 1
#             stack[len(stack)] = Drop()
        stack[len(stack)] = Soft()
        self.fc = nn.Sequential(stack)

AttributeError: 'NoneType' object has no attribute '__name__'

In [15]:
def search_training_hyperparameters():
    lr = {
        "name":"learning rate {10^(-⌊x⌋)",
        "bounds":[x for x in range(1,4)]
    }
    batch_size = {
        "name":"batch size {2^⌊x⌋}",
        "bounds":[x for x in range(6,9)]
    }
    epochs = {
        "name":"epochs {10⌊x⌋}",
        "bounds":[x for x in range(2,6)]
    }
    
    config_space = CS.ConfigurationSpace()
    config_space_dict,config_space_ray = {},{}
    
    #start ConfigSpace API
    config_space.add_hyperparameter(
        CS.UniformFloatHyperparameter(
            lr["name"],
            lr["bounds"][0],
            lr["bounds"][-1],
            log=True
        ))
    config_space.add_hyperparameter(
        CS.CategoricalHyperparameter(
            batch_size["name"], 
            batch_size["bounds"]
        ))
    config_space.add_hyperparameter(
        CS.CategoricalHyperparameter(
            epochs["name"], 
            epochs["bounds"]
        ))
    
    #start Ray Search Space API
    config_space_ray[lr["name"]] = tune.loguniform(lr["bounds"][0],lr["bounds"][-1])
    config_space_ray[batch_size["name"]] = tune.choice(batch_size["bounds"])
    config_space_ray[epochs["name"]] = tune.choice(categories=epochs["bounds"])
    
    #start Dragonfly Search Space API
    param_list = [
        {
            "name": lr["name"], 
            "type": "float", 
            "min": lr["bounds"][0], 
            "max": lr["bounds"][-1]
        },
        {
            "name": batch_size["name"], 
            "type": "discrete_numeric", 
            "items": ":".join([str(2**x) for x in batch_size["bounds"]])
        },
        {
            "name": epochs["name"], 
            "type": "discrete_numeric", 
            "items": ":".join([str(10*x) for x in epochs["bounds"]])
        }
    ]
    
    #start BayesOpt Search Space API
    config_space_dict[lr["name"]] = tune.uniform(lr["bounds"][0],lr["bounds"][-1])
    config_space_dict[batch_size["name"]] = tune.uniform(lower=batch_size["bounds"][0], upper=batch_size["bounds"][-1])
    config_space_dict[epochs["name"]] = tune.uniform(lower=epochs["bounds"][0], upper=epochs["bounds"][-1])
    
    #start Discrete Search Search Space API
    param_dict = {p["name"]:p["bounds"] for p in [lr,batch_size,epochs]}
    
    #start PB2 Space API
    min_max_param_dict = {p["name"]:[p["bounds"][0], p["bounds"][-1]] for p in [lr,batch_size,epochs]}
    
    return config_space_dict